In [26]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from gglm.cnncritic import CNNCritic

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
npz = np.load('./fr_mismatch.npz')
t, stim, mask_spikes_te, u_spk_mean_te, mask_spikes_fr, u_spk_mean_fr  = npz['t'], npz['stim'], npz['mask_spikes_te'], npz['u_spk_mean_te'], npz['mask_spikes_fr'], npz['u_spk_mean_fr']
u_te, u_fr = npz['u_te'], npz['u_fr']
r_te, r_fr = npz['r_te'], npz['r_fr']
u0_ml, eta_vals_ml = npz['glm_u0'], npz['glm_eta']

n_batch_te = mask_spikes_te.shape[1]
n_batch_fr = mask_spikes_fr.shape[1]
mask_spikes = np.concatenate((mask_spikes_te, mask_spikes_fr), axis=1)
n_spikes_te = np.sum(mask_spikes_te, 0)
n_spikes_fr = np.sum(mask_spikes_fr, 0)
mask_spikes_te = torch.from_numpy(mask_spikes_te)
mask_spikes = torch.from_numpy(mask_spikes).double()
# y = np.concatenate((np.ones(mask_spikes_te.shape[1]), np.zeros(mask_spikes_fr.shape[1])))
y = torch.cat((torch.ones(n_batch_te), torch.zeros(n_batch_fr)))
u = torch.from_numpy(np.concatenate((u_te, u_fr), axis=1)).double()
r = torch.from_numpy(np.concatenate((r_te, r_fr), axis=1))

ms = 1

In [37]:
lam_w, num_epochs, lr_g, lr_c, clip = 2.5e2, 120, 2e-2, 1e0, 1e2 # normalized
# clip2 = [0]

# theta = torch.cat([torch.ones(len(tbins) - 1).double() * clip[ii] * 1e-1 for ii in range(len(clip))], dim=0)
# theta = torch.cat([torch.ones(len(tbins)).double() * clip * 1e-1 for ii in range(len(clip))], dim=0)
# theta2 = torch.tensor([1e0])

critic = CNNCritic(n=mask_spikes.shape[0], kernel_size=21)
critic.forward(t, mask_spikes, u, r)

# a0 = critic.transform(t, mask_spikes, u, r).detach().numpy()
# w_distance = critic.train(t, mask_spikes, u, r, y, lam_w, lr=lr_c, num_epochs=800)
# w_distance = -np.array(w_distance)
# a_new = critic.transform(t, mask_spikes, u, r).detach().numpy()

# fig, (ax1, ax2, ax3, ax4) = plt.subplots(figsize=(12, 4), ncols=4)
# ax1.plot(w_distance)
# ti = np.arange(tbins[0], tbins[-1], .1)
# vals = critic.r_kernel.interpolate(ti)
# ax2.plot(ti, vals)
# ax3.hist(a0[y == 1], alpha=0.5)
# ax3.hist(a0[y == 0], alpha=0.5)
# ax4.hist(a_new[y == 1], alpha=0.5)
# ax4.hist(a_new[y == 0], alpha=0.5)

# b = torch.einsum('tka,a->tk', critic.X_r_spk, critic.theta[0:critic.X_r_spk.shape[2]]).detach().numpy()
# b_te = b[:, y == 1]
# b_fr = b[:, y == 0]

# critic.get_conv_r_spikes(t, mask_spikes)

# # critic_conv = critic.r_kernel.convolve_continuous(t, shift_array(mask_spikes, 1, fill_value=False))
# critic_conv = torch.einsum('tka,a->tk', critic.X_r_spk, critic.theta[0:critic.X_r_spk.shape[2]]).detach().numpy()
# critic_conv_te = critic_conv[:, :n_batch_te]
# critic_conv_fr = critic_conv[:, n_batch_te:]

# a = critic.transform(t, mask_spikes, u, r)
# a_te = a[:n_batch_te]
# a_fr = a[n_batch_te:]

# fig, (ax1, ax2, ax3) = plt.subplots(figsize=(15, 5), nrows=3)
# ax2.plot(t, r_te[:, 0])
# ax2.plot(t, r_fr[:, 0])
# ax2.plot(t, critic_conv_te[:, 0])
# ax2.plot(t, critic_conv_fr[:, 0])

torch.Size([200, 2, 1000])


RuntimeError: Expected object of scalar type Double but got scalar type Float for argument #3 'mat1' in call to _th_addmm_

In [22]:
w = critic.cnn_layers[0]